# EcoTravel Agent - Exploração dos Dados

Este notebook demonstra a exploração dos dados de viagens sustentáveis e a configuração inicial do sistema.

In [ ]:
import sys
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar path para imports
sys.path.append('../src')

# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Carregamento e Análise dos Dados de Emissões

In [ ]:
# Carregar dados de emissões
emissoes_df = pd.read_csv('../data/emissoes/emissoes_transporte.csv')

print("Dados de Emissões de Transporte:")
print(emissoes_df)
print(f"\nShape: {emissoes_df.shape}")
print(f"\nInfo:")
emissoes_df.info()

In [ ]:
# Visualizar emissões por modal de transporte
plt.figure(figsize=(12, 6))

# Gráfico de barras para emissões
plt.subplot(1, 2, 1)
bars = plt.bar(emissoes_df['modal_transporte'], emissoes_df['emissao_co2_kg_km'])
plt.title('Emissões de CO2 por Modal de Transporte')
plt.xlabel('Modal de Transporte')
plt.ylabel('Emissão CO2 (kg/km)')
plt.xticks(rotation=45)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.005,
             f'{height:.3f}', ha='center', va='bottom')

# Gráfico de custo vs emissão
plt.subplot(1, 2, 2)
plt.scatter(emissoes_df['custo_medio_km'], emissoes_df['emissao_co2_kg_km'], 
           s=100, alpha=0.7)

# Adicionar labels aos pontos
for i, modal in enumerate(emissoes_df['modal_transporte']):
    plt.annotate(modal, 
                (emissoes_df['custo_medio_km'].iloc[i], 
                 emissoes_df['emissao_co2_kg_km'].iloc[i]),
                xytext=(5, 5), textcoords='offset points')

plt.title('Custo vs Emissão de CO2')
plt.xlabel('Custo Médio (R$/km)')
plt.ylabel('Emissão CO2 (kg/km)')

plt.tight_layout()
plt.show()

## 2. Análise dos Hotéis Sustentáveis

In [ ]:
# Carregar dados de hotéis sustentáveis
with open('../data/avaliacoes/hoteis_sustentaveis.json', 'r', encoding='utf-8') as f:
    hoteis_data = json.load(f)

hoteis_df = pd.DataFrame(hoteis_data['hotéis_sustentáveis'])

print("Hotéis Sustentáveis:")
print(hoteis_df[['nome', 'cidade', 'estado', 'nota_sustentabilidade', 'preço_diária']])
print(f"\nEstatísticas:")
print(hoteis_df[['nota_sustentabilidade', 'preço_diária']].describe())

In [ ]:
# Visualizações dos hotéis
plt.figure(figsize=(15, 5))

# Distribuição por estado
plt.subplot(1, 3, 1)
estado_counts = hoteis_df['estado'].value_counts()
plt.pie(estado_counts.values, labels=estado_counts.index, autopct='%1.1f%%')
plt.title('Distribuição de Hotéis por Estado')

# Nota de sustentabilidade
plt.subplot(1, 3, 2)
plt.bar(hoteis_df['nome'], hoteis_df['nota_sustentabilidade'], color='green', alpha=0.7)
plt.title('Nota de Sustentabilidade por Hotel')
plt.ylabel('Nota (0-10)')
plt.xticks(rotation=45, ha='right')

# Preço vs Sustentabilidade
plt.subplot(1, 3, 3)
plt.scatter(hoteis_df['preço_diária'], hoteis_df['nota_sustentabilidade'], 
           s=100, alpha=0.7, color='green')
plt.title('Preço vs Nota de Sustentabilidade')
plt.xlabel('Preço por Diária (R$)')
plt.ylabel('Nota de Sustentabilidade')

plt.tight_layout()
plt.show()

## 3. Análise de Práticas Sustentáveis

In [ ]:
# Extrair e contar práticas sustentáveis
all_practices = []
for practices in hoteis_df['práticas']:
    all_practices.extend(practices)

practices_counts = pd.Series(all_practices).value_counts()

print("Práticas Sustentáveis Mais Comuns:")
print(practices_counts)

# Visualizar práticas
plt.figure(figsize=(10, 6))
practices_counts.plot(kind='bar', color='forestgreen', alpha=0.8)
plt.title('Frequência de Práticas Sustentáveis nos Hotéis')
plt.xlabel('Práticas Sustentáveis')
plt.ylabel('Número de Hotéis')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Simulação de Cálculos de Pegada de Carbono

In [ ]:
# Importar calculadora de carbono
from tools.carbon_calculator import CarbonCalculator

# Criar instância da calculadora
calc = CarbonCalculator()

# Simulação: São Paulo -> Rio de Janeiro (400 km)
distance = 400
routes = [
    {"name": "SP-RJ", "distance": 400},
    {"name": "SP-BSB", "distance": 1000},
    {"name": "RJ-SSA", "distance": 1200},
    {"name": "SP-Floripa", "distance": 600}
]

# Calcular emissões para diferentes rotas e modais
results = []
for route in routes:
    comparison = calc.compare_transport_modes(route["distance"])
    for comp in comparison:
        results.append({
            "route": route["name"],
            "distance": route["distance"],
            "modal": comp["transport_mode"],
            "emission": comp["total_emission_kg_co2"]
        })

results_df = pd.DataFrame(results)
print("Emissões por Rota e Modal:")
print(results_df.pivot(index='route', columns='modal', values='emission').round(2))

In [ ]:
# Visualizar comparação de emissões
plt.figure(figsize=(12, 8))

# Criar gráfico agrupado
pivot_df = results_df.pivot(index='route', columns='modal', values='emission')

ax = pivot_df.plot(kind='bar', figsize=(12, 6), width=0.8)
plt.title('Emissões de CO2 por Rota e Modal de Transporte')
plt.xlabel('Rota')
plt.ylabel('Emissão CO2 (kg)')
plt.legend(title='Modal de Transporte', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Economia de CO2 escolhendo modal sustentável
savings_data = []
for route in routes:
    comparison = calc.compare_transport_modes(route["distance"])
    most_sustainable = comparison[0]  # Já ordenado por emissão
    least_sustainable = comparison[-1]
    
    savings = least_sustainable["total_emission_kg_co2"] - most_sustainable["total_emission_kg_co2"]
    savings_data.append({
        "route": route["name"],
        "distance": route["distance"],
        "savings_kg_co2": savings,
        "best_modal": most_sustainable["transport_mode"],
        "worst_modal": least_sustainable["transport_mode"]
    })

savings_df = pd.DataFrame(savings_data)
print("\nEconomia de CO2 escolhendo modal mais sustentável:")
print(savings_df)

## 5. Análise do Conteúdo dos Guias

In [ ]:
# Carregar e analisar guia de viagens
with open('../data/guias/guia_sustentavel_brasil.txt', 'r', encoding='utf-8') as f:
    guia_content = f.read()

print(f"Tamanho do guia: {len(guia_content)} caracteres")
print(f"Número de linhas: {len(guia_content.split(chr(10)))}")
print(f"Número de palavras: {len(guia_content.split())}")

# Análise de palavras-chave
keywords = ['sustentável', 'carbono', 'energia', 'local', 'eco', 'ônibus', 'trem', 'avião']

keyword_counts = {}
for keyword in keywords:
    count = guia_content.lower().count(keyword.lower())
    keyword_counts[keyword] = count

print("\nFrequência de palavras-chave no guia:")
for word, count in sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{word}: {count} ocorrências")

In [ ]:
# Visualizar frequência de palavras-chave
plt.figure(figsize=(10, 6))
words = list(keyword_counts.keys())
counts = list(keyword_counts.values())

bars = plt.bar(words, counts, color='lightgreen', alpha=0.8)
plt.title('Frequência de Palavras-Chave no Guia de Viagens Sustentáveis')
plt.xlabel('Palavras-Chave')
plt.ylabel('Número de Ocorrências')
plt.xticks(rotation=45)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Teste Básico do Sistema RAG

In [ ]:
# Tentar importar e testar sistema RAG
try:
    from rag.rag_system import AdvancedRAGSystem
    
    print("Inicializando sistema RAG...")
    rag = AdvancedRAGSystem(data_path="../data")
    
    print("Construindo índices...")
    rag.build_index()
    
    print(f"Sistema construído com sucesso!")
    print(f"Documentos carregados: {len(rag.documents)}")
    print(f"Chunks criados: {len(rag.chunks)}")
    
    # Teste de busca
    query = "Como viajar de São Paulo para Rio de Janeiro de forma sustentável?"
    results = rag.search(query, k=3)
    
    print(f"\nTeste de busca: '{query}'")
    print(f"Resultados encontrados: {len(results)}")
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Score: {result['score']:.3f}")
        print(f"Método: {result['method']}")
        print(f"Conteúdo: {result['content'][:200]}...")
        
except Exception as e:
    print(f"Erro ao testar sistema RAG: {e}")
    print("Certifique-se de que todas as dependências estão instaladas.")

## 7. Resumo da Exploração

### Principais Insights:

1. **Emissões de Transporte**: O trem é o modal mais sustentável (0.041 kg CO2/km), seguido pelo ônibus (0.089 kg CO2/km)

2. **Hotéis Sustentáveis**: Média de nota 8.7/10, com práticas como energia solar sendo mais comuns

3. **Economia de CO2**: Escolher modal sustentável pode economizar 50-100+ kg CO2 em viagens longas

4. **Base de Conhecimento**: Guia abrangente com foco em sustentabilidade e práticas locais

### Próximos Passos:
- Implementar ferramentas de busca e cálculo
- Integrar com APIs de clima e busca web
- Desenvolver agente conversacional
- Criar interface de usuário